# 오차역전파법

## 계산 그래프
- 오르쪽으로 진행하는 것을 순전파 forward propagation 
- 반대의 경우는 backward propagation --> 이후 미분 계산시 중요한 역할<br>
![](img/036.jpg)

### 국소적 계산
- 계산 그래프 방식은 국소적 계산들을 통해 원하는 값을 없는다.
- 자신과 관계된 정보만으로 결과를 출력 가능
- 복잡한 다른 문제는 신경 쓸 필요가 없다 <br>  
### 왜 계산 그래프인가?
- 복잡한 문제를 노드 단위로 단순화 
- 중간 계산결과 저장가능 
- 미분의 효율적 계산 (!)
<br>

![](img/037.jpg)
<br>
## 연쇄법칙 chain rule
- 국소적 미분의 전달 원리 
<br>

### 계산 그래프의 역전파
<br> 

![](img/038.jpg)
<br>
역전파를 통해 미분 값을 효율적으로 구할 수 있다.

### 연쇄법칙이란
- 합성함수 : 2개 이상의 함수로 구성된 함수
- 합성함수의 미분은, 합성함수를 구분하는 각 함수의 미분의 곱으로 나타낼 수 있다. 
dz/dx = dz/dt * dt/dx <br>

### 연쇄법칙과 계산 그래프
<br>

![](img/039.jpg)
![](img/040.jpg)
<br>
역전파는 연쇄법칙의 원리와 같다
<br>

## 역전파 구현

In [1]:
# 곱셈 계층 구현

class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y): #순전파
        self.x = x
        self.y = y
        out = x*y
        
        return out
        
    def backward(self, dout):
        dx = dout * self.y # x와 y를 바꾼다. --> x*y의 경우 
        dy = dout * self.x
        
        return dx, dy

![](img/041.jpg)

In [4]:
apple = 100.0
apple_num = 2.0
tax = 1.1

#계층
mul_apple_layer = MulLayer() # 사과 가격 * 사과개수
mul_tax_layer = MulLayer() # 총 비용 * 세금비율

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

price

220.00000000000003

In [7]:
#역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

dtax ,dapple_num ,dapple

(200.0, 110.00000000000001, 2.2)

In [8]:
# 덧샘 계층 구현
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self,x,y):
        out = x + y 
        return out
    
    def backward(self, dout):
        dx = dout*1
        dy = dout*1
        return dx,dy

![](img/042.jpg)

In [9]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

#레이어 정의
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
apple_orange_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(apple_orange_price,tax)

price


715.0000000000001

In [11]:
#역전파
dprice = 1
dapple_orange_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dapple_orange_price)

dapple, dapple_num = mul_apple_layer.backward(dapple_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)

dtax, dapple_num, dapple, dorange_num, dorange

(650, 110.00000000000001, 2.2, 165.0, 3.3000000000000003)

## 활성화 함수 계층 구현하기

### ReLU 계층 
![](img/043.jpg) <br>
![](img/044.jpg)

In [20]:
# ReLU 순전파 역전파 구현
# forward / backward 는 넘파이 배열을 인수로 받음
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self,x):
        self.mask = (x<=0)
        out = x.copy()
        out[self.mask] = 0 # x<=0 조건에서는 해당인자 1 출력 // 이외 조건은 해당항목 0 출력
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx
        

In [19]:
import numpy as np
x = np.array([[1,-.5],[-2,3]])
mask = (x<=0)
print(x)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]
